In [1]:
!pip install transformers torch nltk

In [2]:
import nltk
import torch
import re
import requests
from nltk.tokenize import word_tokenize
from transformers import GPT2LMHeadModel, GPT2Tokenizer

nltk.download('punkt')
nltk.download('punkt_tab')


c:\Users\avira\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\avira\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\avira\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

LOAD THE GPT-2 MODEL AND SET UP THE ENVIRONMENT

In [3]:
model_name="gpt2"
tokenizer=GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval() #to set to evaluation mode

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

IMPORT AN E-BOOK FROM PROJECT GUTENBERG (IN THIS CASE WE HAVE DOWNLOADED RAMAYANA)


In [6]:
import requests

url = "https://www.gutenberg.org/files/24869/24869-0.txt"  
response = requests.get(url)
data = response.text


TOKENIZING/PRE-PROCESSING THE TEXT

In [7]:
def clean_document(document, char_filter=r"[^\w]"):
    
    document = document.lower()  # Convert to lowercase
    
    words = word_tokenize(document) # Tokenize words
    
    document = " ".join(words) # Remove whitespace and join back words
    
    document = re.sub(char_filter, " ", document) # Remove unwanted characters
    
    document = re.sub(r"\s+", " ", document) # Remove extra whitespace
    
    document = re.sub(r"\s+", " ", document) # Replace multiple whitespaces with a single space

    return document.strip()

data = clean_document(data)

FUNCTION TO DEFINE A TEXT GENERATION USING GPT-2

In [8]:
def generate_text(model,tokenizer,prompt,max_length=100,num_return_sequence=1):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    
    output=model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=num_return_sequence,
        temperature=1.0,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2
    )
    generated_texts=[tokenizer.decode(output,skip_special_tokens=True) for output in output]
    return generated_texts

CALL THE FUNCTION AND GET THE REQUIRED TEXT AS OUTPUT

In [13]:
seed_text="it was a summer evening"
generated_texts=generate_text(model,tokenizer,seed_text,max_length=100)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [29]:
for  text in enumerate(generated_texts, start=1):
    print(f"{text}")

(1, 'it was a summer evening in the city of Bordeaux, France. The young man had been invited to join his father\'s band for an afternoon concert at Stade de la Ronde and he would have enjoyed it if not more than any other night out there on this continent; but as soon as they were all alone together with their friends who came from around Europe—the only people that could be called "friends"—he began singing along like one:\n"I am so glad I\'m')
